In [1]:
import numpy as np
import itertools
import random
from statistics import stdev
import math
import ast

In [2]:
p_running = 0.9
p_failure = 1 - p_running
path = '/home/dkabe/Model_brainstorming/SAA_Analysis_v3/'

In [2]:
N = [50, 100, 200, 300, 400, 500]
N = [1000, 2000]
batches = 30
Products = 3
Market = 29
demand_realization = 25
text_file = open(path + 'Disruption_scenarios.txt', "r")
ls = text_file.read().split('\n')[:-1]
Scenarios = list(map(lambda x: ast.literal_eval(x), ls))
Probabilities = np.loadtxt(path + "disruption_probabilities.txt")
num_disruption = len(Scenarios)


NameError: name 'path' is not defined

In [4]:
p1_mean = 137000
p2_mean = 135000
p3_mean = 133000

p1_sd = 2650
p2_sd = 8125
p3_sd = 4000

distribution_mat = [[p1_mean, p1_sd],
                  [p2_mean, p2_sd],
                  [p3_mean, p3_sd]]

In [5]:
np.random.seed(1)
demand = np.zeros((demand_realization, Products, Market))
for r in range(demand_realization):
    demand[r][0] = np.round(np.random.normal(distribution_mat[0][0], distribution_mat[0][1], size = 29))
    demand[r][1] = np.round(np.random.normal(distribution_mat[1][0], distribution_mat[1][1], size = 29))
    demand[r][2] = np.round(np.random.normal(distribution_mat[2][0], distribution_mat[2][1], size = 29))
#np.savetxt("/home/dkabe/Model_brainstorming/SAA_Analysis_v3/demand.txt", demand.reshape(25*3, 29))

In [6]:
S = demand_realization*Scenarios
S_demand = []
S_prob = list(Probabilities)*demand_realization
multiplier = sum(S_prob)
S_prob = list(map(lambda x: x/multiplier, S_prob))
for r in range(demand_realization):
    S_demand+=([demand[r]]*num_disruption)

In [24]:
random.seed(1)
evaluation_indices = random.sample(range(len(S)), 10000)
evaluation_set = [S[index] for index in evaluation_indices]
evaluation_demand = np.array([S_demand[index] for index in evaluation_indices])
eval_probabilities = [S_prob[index] for index in evaluation_indices]
p_factor = sum(eval_probabilities)
eval_probabilities = list(map(lambda x: x/p_factor, eval_probabilities))
#with open(path + 'Evaluation_Set.txt', 'w') as f:
 #   for item in evaluation_set:
  #      f.write("%s\n" % item)
#f.close()
#np.savetxt(path + "p_scen.txt", eval_probabilities)
#np.savetxt("eval_set_demand.txt", evaluation_demand.reshape((10000*3,29)))

In [64]:
random.seed(1)
for num_scen in N:
    for batch in range(batches):
        indices = random.sample(range(len(S)), num_scen)
        sample_scenarios = [S[index] for index in indices]
        p_sample_scenarios = [S_prob[index] for index in indices]
        demand = np.array([S_demand[index] for index in indices])
        p_factor = 1/np.sum(p_sample_scenarios)
        p_sample_scenarios = list(map(lambda x: x*p_factor, p_sample_scenarios))  
        with open(path + 'Scenarios/' + str(num_scen) + '_' + str(batch) + '.txt', 'w') as f:
            for item in sample_scenarios:
                f.write("%s\n" % item)
        f.close()
        np.savetxt(path + 'Scen_probabilities/' + 'p_scen_' + str(num_scen) + '_' + str(batch) + '.txt', p_sample_scenarios)
        np.savetxt(path + 'Scen_demand/' + 'demand_' + str(num_scen) + '_' + str(batch) + '.txt', demand.reshape((num_scen*Products, Market)))


In [3]:
# version 2
path = '/home/dkabe/Model_brainstorming/SAA_Analysis_v3/'
scenarios = [50, 100, 200, 300, 400, 500, 1000]
p_scen = np.loadtxt('/home/dkabe/Model_brainstorming/SAA_Analysis_v3/p_scen.txt') 

for scen in scenarios:
    gaps = []
    ub_averages = []
    ub_max = []
    objvals_lb = np.loadtxt(path + "Objectives/" + str(scen) + '_results.txt')
    avg_lb = (np.mean(objvals_lb))
    std_lb = (stdev(objvals_lb))
    lbwidth = std_lb/math.sqrt(batches)*2.045
    lbmin = round(avg_lb - lbwidth,2)
    #print(avg_lb, lbmin)
    #print('ci on lower bound = [', round(avg_lb-lbwidth,2), ',', round(avg_lb+lbwidth,2), ']')
    for batch in range(batches):
        objvals_ub = np.loadtxt(path + "Upper_Bounds/" + str(scen) + "_scenarios/" + str(scen) + "_" + str(batch) + "_UB_results.txt")
        avg_ub = (np.sum(p_scen*objvals_ub))
        std_ub = math.sqrt(np.sum(p_scen*((objvals_ub - avg_ub)**2)))
        ubwidth = std_ub/math.sqrt(10000)*1.96
        ubmax = round(avg_ub + ubwidth, 2)
        gap = round(100*((ubmax - lbmin)/lbmin),2)
        ub_averages.append(avg_ub)
        gaps.append(gap)
        ub_max.append(ubmax)
    min_gap = min(filter(lambda x: x >= 0, gaps))
    #min_gap = min(gaps)
    index = gaps.index(min_gap)
    #print(ub_averages[index], ub_max[index], min_gap)
    print(avg_lb, "\t", ub_averages[index], "\t", lbmin, "\t", ub_max[index], "\t", min_gap)

19092303.575999998 	 18737033.4819952 	 17936816.26 	 18850730.16 	 5.1
19707049.249 	 18737033.4819952 	 18634623.6 	 18850730.16 	 1.16
18791822.872666664 	 18737033.4819952 	 18153017.77 	 18850730.16 	 3.84
19478004.370666668 	 18899003.142476276 	 18927827.77 	 19017324.6 	 0.47
19094568.076333337 	 18737033.4819952 	 18600351.07 	 18850730.16 	 1.35
19394766.26133333 	 18899003.142476276 	 18867826.51 	 19017324.6 	 0.79
18974217.647 	 18737033.4819952 	 18718395.54 	 18850730.16 	 0.71


In [18]:
gaps

[1.9,
 0.79,
 8.73,
 -0.09,
 1.6,
 1.9,
 1.6,
 0.79,
 1.9,
 -0.09,
 3.84,
 -0.09,
 -0.09,
 1.17,
 0.79,
 1.9,
 1.17,
 15.53,
 11.88,
 1.6,
 -0.09,
 6.35,
 -0.09,
 -0.09,
 -0.09,
 1.17,
 -0.09,
 6.77,
 8.28,
 -0.09]

In [18]:
gaps

[1.9,
 0.79,
 8.73,
 -0.09,
 1.6,
 1.9,
 1.6,
 0.79,
 1.9,
 -0.09,
 3.84,
 -0.09,
 -0.09,
 1.17,
 0.79,
 1.9,
 1.17,
 15.53,
 11.88,
 1.6,
 -0.09,
 6.35,
 -0.09,
 -0.09,
 -0.09,
 1.17,
 -0.09,
 6.77,
 8.28,
 -0.09]